## Goal 
### glue together many df.

### Stats
+ What are summary statistics of each feature?
+ How has thanking changed over time?
+ How is gratitude distributes among users?
+ What do the distirbutions of each feature look like?
+ For multi-window features how do they look together?

In [1]:
import pyarrow as pa
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import os
# from collections import defaultdict
%pylab inline
import json
from datetime import datetime as dt
nat_filler = dt(1970,1,1)
import gc

Populating the interactive namespace from numpy and matplotlib


In [2]:
langcodes = json.load(open('configs/ores.json','r'))['langcodes']

In [3]:
outputs_dir = 'outputs'
file_list = os.listdir(outputs_dir)

In [4]:
numericals = ['receiver_prev_edits', 'sender_prev_edits', 'sender_edits_1d_after', 'sender_edits_30d_after', 'sender_edits_90d_after', 'sender_edits_180d_after', 'receiver_edits_1d_after', 'receiver_edits_30d_after', 'receiver_edits_90d_after', 'receiver_edits_180d_after', 'receiver_prev_received', 'receiver_prev_sent', 'sender_prev_received', 'sender_prev_sent', 'receiver_thank_another_1d_after', 'receiver_thank_another_30d_after', 'receiver_thank_another_90d_after', 'receiver_thank_another_180d_after']

In [5]:
# ## Sub step Fix Enwiki file first.
# # There may be some negative -1's because had to subtract 1 in each calculation 
# # because of inclusive indices different when useing datetime index or the standard index,
# # but if the selection was empty we'd have a result of 0-1. 

# enthank_file = 'outputs/wikithank_en_20180804.csv'
# # enthank_file = 'outputs/wikilove_en_20180805.csv'
# endf = pd.read_csv(enthank_file)

# endf.columns

# numericals = ['receiver_prev_edits', 'sender_prev_edits', 'sender_edits_1d_after', 'sender_edits_30d_after', 'sender_edits_90d_after', 'sender_edits_180d_after', 'receiver_edits_1d_after', 'receiver_edits_30d_after', 'receiver_edits_90d_after', 'receiver_edits_180d_after', 'receiver_prev_received', 'receiver_prev_sent', 'sender_prev_received', 'sender_prev_sent', 'receiver_thank_another_1d_after', 'receiver_thank_another_30d_after', 'receiver_thank_another_90d_after', 'receiver_thank_another_180d_after']

# def show_negs():
#     tot = len(endf)
#     for n in numericals:
#         ser = endf[n]
#         neg = ser < 0
#         print(n)
#         print(len(ser[neg]) / tot)
#     #     print(ser.head(5))
#     #     print(ser[neg].head(5))
#         print(ser.min())

# show_negs()

# endf.replace(to_replace=-1, value=0, inplace=True)

# show_negs()

# #back enfile
# !cp $enthank_file $enthank_file'.bak'
# #save new enfile
# endf.to_csv(enthank_file)

In [6]:
files = dict()
for f in file_list:
    parts = f.split('_')
    thanklove = parts[0].split('wiki')[1]
    lang = parts[1]

    tdf = pd.read_csv(os.path.join(outputs_dir, f), parse_dates=['timestamp', 'receiver_first_edit', 'sender_first_edit'])
    if thanklove == 'love':
        # i will fix this upstream so this shouldn't have to be done in the future.
        tdf['wll_type'] = tdf['wll_type'].apply(lambda x: x.replace("b'","").replace("'",""))
    
    #feather requires homogenous datatypes
    tdf['receiver'] = tdf['receiver'].astype(str)
    tdf['sender'] = tdf['sender'].astype(str)
    
    tdf['thanklove'] = thanklove
    tdf['lang'] = lang
    files[f] = tdf
#print(files)

In [7]:
df = pd.concat(files.values())
df = df.reset_index(drop=True)
#df = df.fillna({'sender_first_edit':nat_filler, 'receiver_first_edit':nat_filler})
df['probably_deleted'] = (pd.isna(df['sender_first_edit'])) | (pd.isna(df['receiver_first_edit']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
nulldf = df[df[numericals].isnull().any(axis=1)]

In [9]:
nulldf.shape

(1047, 34)

In [10]:
for n in numericals:
    df[n] = df[n].fillna(0).astype(int)

In [11]:
mapper = {'thanklove':'gratitude',
         'receiver_thank_another_1d_after':'receiver_gratitude_another_1d_after',
         'receiver_thank_another_30d_after':'receiver_gratitude_another_30d_after',
         'receiver_thank_another_90d_after':'receiver_gratitude_another_90d_after',
         'receiver_thank_another_180d_after':'receiver_gratitude_another_180d_after',
            }
df.rename(mapper, axis='columns', inplace=True)

In [12]:
#redo column order
new_order = ['lang',
            'timestamp',
             'gratitude',
             'receiver',
            'sender',
            'receiver_id',
            'sender_id',
             'wll_type',
            'probably_deleted',

            'receiver_prev_edits',
            'receiver_prev_received',
            'receiver_prev_received_indicator',
            'receiver_prev_sent',
            'receiver_prev_sent_indicator',

             
            'receiver_edits_1d_after',
            'receiver_edits_30d_after',
            'receiver_edits_90d_after',
            'receiver_edits_180d_after',

            
            'receiver_gratitude_another_1d_after',
            'receiver_gratitude_another_30d_after',
            'receiver_gratitude_another_90d_after',
            'receiver_gratitude_another_180d_after',

            'receiver_first_edit',
            
            'sender_prev_edits',
            'sender_prev_received',
            'sender_prev_received_indicator',
            'sender_prev_sent',
            'sender_prev_sent_indicator',

            
            'sender_edits_1d_after',
            'sender_edits_30d_after',
            'sender_edits_90d_after',
            'sender_edits_180d_after',
             
            'sender_first_edit',
            ]

df = df[new_order]

In [13]:
today = dt.today().strftime("%Y%m%d")

In [14]:
len(df)

3849806

In [15]:
df.to_feather(f'transfer/gratitude_{today}.feather')

In [16]:
import math

In [29]:
def write_cs_rows(df, chunkrows, fname):
    # delete file
    try:
        os.remove(fname)
    except FileNotFoundError:
        pass
    # write header
    df[:0].to_csv(fname, index=False)
    # write chunks in loop with mode = a
    required_chunks = math.ceil(len(df)/chunkrows)
    for i in range(required_chunks):
        startind = i*chunkrows
        stopind = ((i+1)*chunkrows)
        partdf = df.iloc[startind:stopind]
        print(len(partdf))
        partdf.to_csv(fname, mode='a', header=False)
        gc.collect()
        print(stopind)
#         print(df.iloc[startind:stopind]['timestamp'].head(1))

# call gc.collect()

write_cs_rows(df, 100000, f'transfer/gratitude_{today}.csv')

100000
100000
100000
200000
100000
300000
100000
400000
100000
500000
100000
600000
100000
700000
100000
800000
100000
900000
100000
1000000
100000
1100000
100000
1200000
100000
1300000
100000
1400000
100000
1500000
100000
1600000
100000
1700000
100000
1800000
100000
1900000
100000
2000000
100000
2100000
100000
2200000
100000
2300000
100000
2400000
100000
2500000
100000
2600000
100000
2700000
100000
2800000
100000
2900000
100000
3000000
100000
3100000
100000
3200000
100000
3300000
100000
3400000
100000
3500000
100000
3600000
100000
3700000
100000
3800000
49806
3900000


In [36]:
len(df)

3849806

In [35]:
dfconfirm = pd.read_csv(f'transfer/gratitude_{today}.csv')

KeyboardInterrupt: 

In [20]:
len(dfconfirm)

3849806

In [21]:
df.tail(3)

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
3849803,tr,2018-05-30 21:47:19,thank,Pragdon,Sakhalinio,272157,86010,NaN,False,20660,288,True,270,True,19,639,639,639,0,0,0,0,2012-02-09 23:52:29,87764,1092,True,3173,True,18,1024,1024,1024,2008-08-22 08:24:19
3849804,tr,2018-05-31 19:13:07,thank,Te750iv,Sakhalinio,974240,86010,NaN,False,4,0,False,0,False,0,0,0,0,0,0,0,0,2017-10-16 10:56:50,87774,1092,True,3174,True,11,1014,1014,1014,2008-08-22 08:24:19
3849805,tr,2018-05-31 19:18:14,thank,Ascoltami,Sakhalinio,986217,86010,NaN,False,8,0,False,0,False,1,1,1,1,0,0,0,0,2017-12-12 23:08:28,87779,1092,True,3175,True,6,1009,1009,1009,2008-08-22 08:24:19


In [22]:
dfconfirm.tail(3)

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
3849803,tr,2018-05-30 21:47:19,thank,Pragdon,Sakhalinio,272157,86010,NaN,False,20660,288,True,270,True,19,639,639,639,0,0,0,0,2012-02-09 23:52:29,87764,1092,True,3173,True,18,1024,1024,1024,2008-08-22 08:24:19
3849804,tr,2018-05-31 19:13:07,thank,Te750iv,Sakhalinio,974240,86010,NaN,False,4,0,False,0,False,0,0,0,0,0,0,0,0,2017-10-16 10:56:50,87774,1092,True,3174,True,11,1014,1014,1014,2008-08-22 08:24:19
3849805,tr,2018-05-31 19:18:14,thank,Ascoltami,Sakhalinio,986217,86010,NaN,False,8,0,False,0,False,1,1,1,1,0,0,0,0,2017-12-12 23:08:28,87779,1092,True,3175,True,6,1009,1009,1009,2008-08-22 08:24:19


In [23]:
df[99998:100003]['receiver']

99998                     Milez189
99999                    Henrik.01
100000                Vlad2000Plus
100001    Vbngrbsknhobnpgrnbongonj
100002                   Maas Leag
Name: receiver, dtype: object

In [24]:
dfconfirm[99998:100003]

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
99998,ru,2016-02-28 20:39:15,thank,Milez189,Roundabout,1290702,429898,NaN,False,39674,290,True,97,True,153,6253,16153,16168,0,1,18,18,2014-07-14 10:46:21,8780,18,True,40,True,9,203,487,724,2010-02-11 22:08:08
99999,ru,2016-02-28 21:03:47,thank,Henrik.01,Mahno30,1164403,1864455,NaN,True,2,0,False,0,False,0,0,0,0,0,0,0,0,2015-03-22 06:54:10,0,0,False,0,False,0,0,0,0,NaN
100000,ru,2016-02-28 21:03:58,thank,Vlad2000Plus,KartMan,8414,1864434,NaN,True,11888,53,True,0,False,29,1671,1787,1991,0,1,1,1,2006-01-03 01:01:44,0,0,False,0,False,0,0,0,0,NaN
100001,ru,2016-02-28 21:12:37,thank,Vbngrbsknhobnpgrnbongonj,Повелитель Звёзд,1295384,1346505,NaN,False,31,0,False,0,False,0,2,3,3,0,0,0,0,2014-08-09 18:15:52,2477,24,True,861,True,23,486,1489,2538,2014-09-03 00:06:21
100002,ru,2016-02-28 21:20:19,thank,Maas Leag,Anji-dag,1775212,1144114,NaN,False,51,0,False,0,False,0,11,190,191,0,0,0,0,2015-10-24 07:11:06,470,0,False,14,True,0,7,60,64,2013-11-12 17:23:41


In [25]:
!head 'transfer/gratitude_20180813.csv'

lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
0,ar,2011-10-12 20:38:38,love,341772,341772,341772,341772,barnstar-original,False,2,0,False,0,False,0,0,0,1,0,0,0,0,2011-10-12 20:35:46,2,0,False,0,False,0,0,0,1,2011-10-12 20:35:46
1,ar,2011-10-13 11:56:45,love,418003,418003,418003,418003,kitten,False,0,0,False,0,False,1,1,1,1,0,0,0,0,2011-10-

In [26]:
df[:0]

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit


In [27]:
df.head()

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
0,ar,2011-10-12 20:38:38,love,341772,341772,341772,341772,barnstar-original,False,2,0,False,0,False,0,0,0,1,0,0,0,0,2011-10-12 20:35:46,2,0,False,0,False,0,0,0,1,2011-10-12 20:35:46
1,ar,2011-10-13 11:56:45,love,418003,418003,418003,418003,kitten,False,0,0,False,0,False,1,1,1,1,0,0,0,0,2011-10-13 11:56:46,0,0,False,0,False,1,1,1,1,2011-10-13 11:56:46
2,ar,2011-10-13 12:19:32,love,411973,411973,411973,411973,barnstar-original,False,1,0,False,0,False,1,1,1,1,1,1,1,1,2011-10-13 12:14:15,1,0,False,0,False,1,1,1,1,2011-10-13 12:14:15
3,ar,2011-10-13 12:19:47,love,411973,411973,411973,411973,barnstar-original,False,2,1,True,1,True,1,1,1,1,0,0,0,0,2011-10-13 12:14:15,2,1,True,1,True,1,1,1,1,2011-10-13 12:14:15
4,ar,2011-10-13 13:26:24,love,334655,334655,334655,334655,barnstar-original,False,437,0,False,0,False,4,38,91,91,1,1,1,1,2010-12-11 18:40:48,437,0,False,0,False,4,38,91,91,2010-12-11 18:40:48


In [37]:
del dfconfirm

NameError: name 'dfconfirm' is not defined

In [38]:
!du -h transfer/*

OSError: [Errno 12] Cannot allocate memory

In [40]:
df[df['lang']=='en'].tail()

,lang,timestamp,gratitude,receiver,sender,receiver_id,sender_id,wll_type,probably_deleted,receiver_prev_edits,receiver_prev_received,receiver_prev_received_indicator,receiver_prev_sent,receiver_prev_sent_indicator,receiver_edits_1d_after,receiver_edits_30d_after,receiver_edits_90d_after,receiver_edits_180d_after,receiver_gratitude_another_1d_after,receiver_gratitude_another_30d_after,receiver_gratitude_another_90d_after,receiver_gratitude_another_180d_after,receiver_first_edit,sender_prev_edits,sender_prev_received,sender_prev_received_indicator,sender_prev_sent,sender_prev_sent_indicator,sender_edits_1d_after,sender_edits_30d_after,sender_edits_90d_after,sender_edits_180d_after,sender_first_edit
3580841,en,2018-05-31 23:56:04,thank,Dilidor,Toughpigs,25648301,578047,NaN,False,8404,76,True,17,True,20,261,572,572,0,0,0,0,2015-06-30 19:18:29,7571,35,True,5,True,72,149,171,171,2005-11-14 01:46:31
3580842,en,2018-05-31 23:56:22,thank,Kyteto,Ahunt,9218062,253891,NaN,False,22861,125,True,0,False,1,288,354,354,0,0,0,0,2009-03-15 01:40:45,145639,2029,True,7348,True,48,902,1131,1131,2005-04-30 21:23:09
3580843,en,2018-05-31 23:56:35,thank,Haploidavey,InformationvsInjustice,8882122,16692328,NaN,False,33084,166,True,94,True,24,335,508,508,0,0,0,0,2009-02-02 19:35:30,2748,13,True,297,True,3,134,192,192,2012-04-22 03:13:04
3580844,en,2018-05-31 23:57:10,thank,Acroterion,InformationvsInjustice,1839637,16692328,NaN,False,146071,1511,True,217,True,36,991,1409,1409,0,0,0,0,2006-07-22 01:31:04,2748,13,True,298,True,3,134,192,192,2012-04-22 03:13:04
3580845,en,2018-05-31 23:57:44,thank,Pegship,Jjjjjjdddddd,355698,20083999,NaN,False,93754,88,True,35,True,1,119,146,146,0,0,0,0,2005-07-29 19:29:33,4389,48,True,80,True,5,58,58,58,2013-11-06 04:54:04


In [41]:
def frac_nan(df):
    sender_nan = df[pd.isna(df['sender_first_edit'])]
    receiver_nan = df[pd.isna(df['receiver_first_edit'])]
    either_nan = df[(pd.isna(df['sender_first_edit'])) | (pd.isna(df['receiver_first_edit']))]
    denom = len(df)
    sender_nan_frac = len(sender_nan) / denom
    receiver_nan_frac = len(receiver_nan) / denom
    either_nan_frac = len(either_nan) / denom
    return {'sender_nan_frac': sender_nan_frac,
            'receiver_nan_frac': receiver_nan_frac,
            'either_nan_frac': either_nan_frac}

In [42]:
frac_nan(df)

{'either_nan_frac': 0.012047360308545418,
 'receiver_nan_frac': 0.0025432450362433846,
 'sender_nan_frac': 0.009862315140035627}

In [43]:
nan_fracs = {}
for fname, df in files.items():
    nan_fracs[fname] = frac_nan(df)
nan_frac_df = pd.DataFrame.from_dict(nan_fracs, orient='index')

In [44]:
nan_frac_df.sort_values('either_nan_frac')

,sender_nan_frac,receiver_nan_frac,either_nan_frac
wikithank_he_20180627.csv,0.001004,0.000854,0.001858
wikithank_de_20180712.csv,0.002218,0.000606,0.002805
wikithank_et_20180704.csv,0.002881,0.000206,0.003087
wikithank_no_20180703.csv,0.002247,0.001324,0.003170
wikithank_hu_20180627.csv,0.003014,0.000328,0.003343
wikithank_sv_20180628.csv,0.003561,0.000838,0.004150
wikilove_uk_20180703.csv,0.001439,0.002878,0.004317
wikithank_cs_20180704.csv,0.003793,0.000722,0.004392
wikithank_hr_20180702.csv,0.004275,0.000570,0.004845
wikithank_uk_20180703.csv,0.004158,0.000715,0.004860


In [45]:
nan_frac_df.mean()

sender_nan_frac      0.017438
receiver_nan_frac    0.006289
either_nan_frac      0.022120
dtype: float64

In [46]:
df_filled = df.fillna(nat_filler)

In [47]:
frac_nan(df_filled)

{'either_nan_frac': 0.0, 'receiver_nan_frac': 0.0, 'sender_nan_frac': 0.0}